# Function Calling 函数使用

In [ ]:
## （一）定义函数
# 在使用 Function Calling 之前，开发者需要定义可供模型调用的函数。这些函数需要有清晰的定义，包括函数名、参数列表和功能描述。
# 例如，定义一个获取城市天气的函数：

In [39]:
def get_weather(city):
    """
    获取指定城市的天气信息
    :param city: 城市名称
    :return: 包含天气状况和温度的字典
    """
    # 这里是实际获取天气数据的代码，例如调用天气API
    weather_data = {
        "condition": "sunny",
        "temperature": "25°C"
    }
    return weather_data


In [ ]:
## （二）初始化大模型客户端
## 导入相关的依赖包，并初始化大模型客户端

In [40]:
import json
from openai import OpenAI

client = OpenAI(api_key="sk-47a1288c898f43ecbb41864bf8f85a8a", base_url="https://api.deepseek.com")

In [ ]:
## （三）定义tools工具函数

In [41]:
# 定义tools工具函数
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "获取指定城市的天气信息",
            "parameters": {
                "type": "object",
                "properties": {
                    "city": {
                        "type": "string",
                        "description": "城市名称"
                    }
                },
                "required": ["city"]
            }
        }
    }
]


In [ ]:
## （四）第一次调用大模型API

In [42]:
# 定义消息
messages = [
    {"role": "user", "content": "明天北京的天气如何"}
]



# 第1次大模型调用（查看是否有匹配到工具函数）
response = client.chat.completions.create(
    model="deepseek-chat",
    messages=messages,
    tools=tools,
    tool_choice="auto"
)
# 查看第一次调用后返回的消息（检查大模型是否有找到函数信息）
reply = response.choices[0].message
messages.append(reply)
print(reply)

ChatCompletionMessage(content='', role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_0_a8dfb1f7-7718-4584-9e9f-0ba520fa49dd', function=Function(arguments='{"city":"北京"}', name='get_weather'), type='function', index=0)])


In [ ]:
## （五）调用真正的业务函数

In [43]:
function_args = json.loads(reply.tool_calls[0].function.arguments)  # 确保转换为字典
result = get_weather(**function_args) #真正调用，手动执行获取天气信息的方法
print(result)

{'condition': 'sunny', 'temperature': '25°C'}


In [ ]:
## （六）第二次调用大模型API
# 第2次大模型调用；将获取到的结果，丢给大模型帮忙整理后再输出给用户。

In [44]:
# messages.append({"role": "tool", "name": function_name, "content": str(result)})
tool_call_id = reply.tool_calls[0].id
#print("tool_call_id",tool_call_id)
messages.append({"role": "tool", "tool_call_id": tool_call_id, "content": str(result)})
second_response = client.chat.completions.create(
    model="deepseek-chat",
    messages=messages
)
print(second_response.choices[0].message.content)

明天北京的天气预计是晴天，气温大约在25°C左右。记得做好防晒措施哦！


In [ ]:
## 基于function calling自助查询天气，完整代码如下：

In [38]:
# 1、导入依赖&初始化大模型客户端
import json
from openai import OpenAI

client = OpenAI(api_key="sk-47a1288c898f43ecbb41864bf8f85a8a", base_url="https://api.deepseek.com")

# 2、定义消息
messages = [
    {"role": "user", "content": "明天北京的天气如何"}
]

# 3、定义tools工具函数
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "获取指定城市的天气信息",
            "parameters": {
                "type": "object",
                "properties": {
                    "city": {
                        "type": "string",
                        "description": "城市名称"
                    }
                },
                "required": ["city"]
            }
        }
    }
]


# 第1次大模型调用（查看是否有匹配到工具函数）
response = client.chat.completions.create(
    model="deepseek-chat",
    messages=messages,
    tools=tools,
    tool_choice="auto"
)
# 查看第一次调用后返回的消息（检查大模型是否有找到函数信息）
reply = response.choices[0].message
print(reply)
messages.append(reply)

if reply.tool_calls:
    #function_name = reply.tool_calls[0].function.name
    #function_args = reply.tool_calls[0].function.arguments
    function_args = json.loads(reply.tool_calls[0].function.arguments)  # 确保转换为字典
    result = get_weather(**function_args) #真正调用，手动执行获取天气信息的方法
    print(result)
    # 第2次大模型调用（将获取到的结果，丢给大模型帮忙整理后再输出给用户）
    # messages.append({"role": "tool", "name": function_name, "content": str(result)})
    tool_call_id = reply.tool_calls[0].id
    #print("tool_call_id",tool_call_id)
    messages.append({"role": "tool", "tool_call_id": tool_call_id, "content": str(result)})
    second_response = client.chat.completions.create(
        model="deepseek-chat",
        messages=messages
    )
    print(second_response.choices[0].message.content)
else:
    print(reply.message.content)


ChatCompletionMessage(content='', role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_0_89902ab0-a987-47c9-9d3b-d1a4913df82a', function=Function(arguments='{"city":"北京"}', name='get_weather'), type='function', index=0)])
{'condition': 'sunny', 'temperature': '25°C'}
根据最新的天气预报，明天北京的天气将会是晴天，气温大约在25°C左右。这样的天气非常适合外出活动，记得做好防晒措施哦！如果你有其他问题或需要更多信息，随时告诉我！
